In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col,lit
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

df.show()
# df.join(df, lit(1)==lit(1)).take(5)

24/12/06 13:12:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|2021-09-26 00:00:00|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|2021-02-21 00:00:00|
| -824540328| 532630305|    NULL|admi

In [2]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host")) \
           .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.show()
 # col("browser_family")

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+
|    user_id|  device_id|            referrer|                host|                 url|          event_time|         event_date|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+
| 1129583063|  532630305|                NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|
| -648945006| 1088283544|                NULL|    www.eczachly.com|                   /|2021-01-07 02:58:...|2021-01-07 00:00:00|
|-1871780024| -158310583|                NULL|    www.eczachly.com|                   /|2021-01-07 04:17:...|2021-01-07 00:00:00|
|  203689086| 1088283544|                NULL|    www.eczachly.com|/blog/what-exactl...|2021-01-07 10:03:...|2021-01-07 00:00:00|
|-1180485268|  532630305|                NULL|    www.eczachly.com|                   /|20

In [3]:
sortedTwo = df.repartition(10, col("event_date")) \
        .sort(col("event_date"), col("host")) \
           .withColumn("event_time", col("event_time").cast("timestamp")) 
sortedTwo.show()

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+
|    user_id|  device_id|            referrer|                host|                 url|          event_time|         event_date|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+
| 1272828233| -643696601|                NULL|admin.zachwilson....|                   /|2021-01-02 13:53:...|2021-01-02 00:00:00|
|  747494706|  532630305|                NULL|admin.zachwilson....|                   /|2021-01-02 19:36:...|2021-01-02 00:00:00|
| 2110046626|  898871897|                NULL|admin.zachwilson....|       /wp-login.php|2021-01-02 19:57:...|2021-01-02 00:00:00|
| 1272828233| -643696601|                NULL|admin.zachwilson....|                   /|2021-01-02 21:05:...|2021-01-02 00:00:00|
| 1272828233| -643696601|                NULL|admin.zachwilson....|                   /|20

In [5]:
sorted.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#73, event_date#29]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 100), REPARTITION_BY_NUM, [plan_id=132]
         +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
            +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,host:string,url:string,event_time:string>




In [6]:
sortedTwo.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#116, event_date#29]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], true, 0
      +- Exchange rangepartitioning(event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=154]
         +- Exchange hashpartitioning(event_date#29, 100), REPARTITION_BY_NUM, [plan_id=150]
            +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,ho

in the .explain:
Project and select mean the same thing. </br>
Exchange hashpartitioning does the partitioning. </br>
In the sort line, there is a boolean parameter which tells us if the sort is global or not. </br>
The Exchange rangepartitioning is the painful part (at scale). </br>
Exchange means shuffle!

In [3]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [4]:
%%sql 
SHOW TABLES IN bootcamp;

namespace,tableName,isTemporary
bootcamp,events,False
bootcamp,events_sorted,False
bootcamp,events_unsorted,False


In [5]:
%%sql
DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [2]:
%%sql
DROP TABLE IF EXISTS bootcamp.events_sorted

Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/bootcamp/events_sorted/metadata/00000-dd872742-0436-40c0-ae4d-05eed17b4355.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:201)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:313)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:252)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:348)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:331)
	at org.apache.iceberg.re

In [11]:
-- %%sql

-- CREATE TABLE IF NOT EXISTS bootcamp.events (
--     url STRING,
--     referrer STRING,
--     browser_family STRING,
--     os_family STRING,
--     device_family STRING,
--     host STRING,
--     event_time TIMESTAMP,
--     event_date DATE
-- )
-- USING iceberg
--     PARTITIONED BY (event_date);
-- PARTITIONED BY (years(event_date));


++
||
++
++

In [6]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    user_id NUMERIC,
    device_id NUMERIC,
    referrer STRING,
    host STRING,
    url STRING,

    event_time TIMESTAMP,
    event_date DATE
    --browser_family STRING,
    --os_family STRING,
    --device_family STRING,
)
USING iceberg
    PARTITIONED BY (event_date);

++
||
++
++

In [2]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    user_id NUMERIC,
    device_id NUMERIC,
    referrer STRING,
    host STRING,
    url STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
    PARTITIONED BY (event_date);
    -- url STRING,
    -- referrer STRING,
    -- browser_family STRING,
    -- os_family STRING,
    -- device_family STRING,
    -- host STRING,
    -- event_time TIMESTAMP,
    -- event_date DATE

++
||
++
++

In [3]:
%%sql
CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    user_id NUMERIC,
    device_id NUMERIC,
    referrer STRING,
    host STRING,
    url STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
    PARTITIONED BY (event_date);

++
||
++
++

In [4]:
start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) 
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host")) #col("browser_family"),

# sorted = df.repartition(10, col("event_date")) \
#         .sortWithinPartitions(col("event_date")) \
#         .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [5]:
%%sql
select * from demo.bootcamp.events_sorted.files
-- .files is the metadata

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/bootcamp/events_sorted/data/00000-22-dad7b32f-1d42-4e8d-a90d-792cb8eaba2c-0-00001.parquet,PARQUET,1,Row(event_date=None),89391,998079,"{1: 315033, 2: 103859, 3: 68766, 4: 2674, 5: 111517, 6: 390792, 7: 2274}","{1: 89391, 2: 89391, 3: 89391, 4: 89391, 5: 89391, 6: 89391, 7: 89391}","{1: 1, 2: 0, 3: 46359, 4: 0, 5: 0, 6: 0, 7: 0}",{},"{1: bytearray(b'-1000095488'), 2: bytearray(b'-100210680'), 3: bytearray(b'52.20.78.240'), 4: bytearray(b'aashish.techcrea'), 5: bytearray(b'/'), 6: bytearray(b' \xba\xe7\xb8\xa8\xb8\x05\x00'), 7: bytearray(b'\x00\xa0&\xb4\xa8\xb8\x05\x00')}","{1: bytearray(b'999884938'), 2: bytearray(b'999535123'), 3: bytearray(b'zachwilson.tech'), 4: bytearray(b'zachwilson.techd'), 5: bytearray(b'/zzageqnf.php?Fp'), 6: bytearray(b'\xe8\xb0\x1b\x8ec\x03\x06\x00'), 7: bytearray(b'\x00\xe0dqO\x03\x06\x00')}",None,[4],None,0,"Row(device_id=Row(column_size=103859, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='-100210680', upper_bound='999535123'), event_date=Row(column_size=2274, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 0), upper_bound=datetime.datetime(2023, 8, 20, 0, 0)), event_time=Row(column_size=390792, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 1, 19, 764000), upper_bound=datetime.datetime(2023, 8, 20, 23, 59, 41, 89000)), host=Row(column_size=2674, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='aashish.techcrea', upper_bound='zachwilson.techd'), referrer=Row(column_size=68766, value_count=89391, null_value_count=46359, nan_value_count=None, lower_bound='52.20.78.240', upper_bound='zachwilson.tech'), url=Row(column_size=111517, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='/', upper_bound='/zzageqnf.php?Fp'), user_id=Row(column_size=315033, value_count=89391, null_value_count=1, nan_value_count=None, lower_bound='-1000095488', upper_bound='999884938'))"
0,s3://warehouse/bootcamp/events_sorted/data/00001-23-dad7b32f-1d42-4e8d-a90d-792cb8eaba2c-0-00001.parquet,PARQUET,1,Row(event_date=None),99232,1126501,"{1: 344927, 2: 117090, 3: 73756, 4: 3294, 5: 145848, 6: 435861, 7: 2355}","{1: 99232, 2: 99232, 3: 99232, 4: 99232, 5: 99232, 6: 99232, 7: 99232}","{1: 58, 2: 0, 3: 49299, 4: 0, 5: 0, 6: 0, 7: 0}",{},"{1: bytearray(b'-1000370060'), 2: bytearray(b'-100210680'), 3: bytearray(b'""https://www.goo'), 4: bytearray(b'abhishekanand.te'), 5: bytearray(b'""/?""""<?=print(93'), 6: bytearray(b'(\x83\xb2EX\xb8\x05\x00'), 7: bytearray(b'\x00 \xc9<X\xb8\x05\x00')}","{1: bytearray(b'999956796'), 2: bytearray(b'999535123'), 3: bytearray(b'zachwilson.tech'), 4: bytearray(b'zsavi524.techcrf'), 5: bytearray(b'/zz.php'), 6: bytearray(b'\x88\xb8\x07P;\x03\x06\x00'), 7: bytearray(b""\x00 \xb65\'\x03\x06\x00"")}",None,[4],None,0,"Row(device_id=Row(column_size=117090, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound='-100210680', upper_bound='999535123'), event_date=Row(column_size=2355, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 8, 0, 0), upper_bound=datetime.datetime(2023, 8, 18, 0, 0)), event_time=Row(column_size=435861, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 8, 0, 2, 29, 513000), upper_bound=datetime.datetime(2023, 8, 18, 23, 59, 0, 901000)), host=Row(column_size=3294, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound='abhishekanand.te', upper_bound='zsavi524.techcrf'), referrer=Row(column_size=73756, value_count=99232, null_value_count=49299, nan_value_count=None, lower_bound='""https://www.goo', upper_bound='za

In [6]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
4955502,4,sorted
5050842,4,unsorted


In [7]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `demo`.`bootcamp`.`events`.`files` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 67;
'Aggregate ['SUM('file_size_in_bytes) AS size#267, count(1) AS num_files#268L]
+- 'UnresolvedRelation [demo, bootcamp, events, files], [], false


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
